In [ ]:
import json
import cv2
import os
import numpy as np

# Define the dataset paths
dataset_path = '/mnt/data/coco_wholebody_dataset/'  # Path to the directory containing the images
output_path = '/mnt/data/augmented_coco_wholebody/'  # Path to save the augmented images
annotations_file = os.path.join(dataset_path, 'annotations.json')  # Path to the annotation file

# Create output directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Function to generate and overlay a simple VR headset on an image based on eye coordinates
def generate_vr_headset(image, eye_coords):
    # Calculate the distance between the eyes
    eye_distance = np.linalg.norm(np.array(eye_coords[0]) - np.array(eye_coords[1]))

    # Define the VR headset dimensions based on the eye distance
    headset_width = int(1.5 * eye_distance)
    headset_height = int(eye_distance * 0.5)
    headset_thickness = 2

    # Calculate the position to draw the VR headset
    center_x = int((eye_coords[0][0] + eye_coords[1][0]) / 2)
    center_y = int((eye_coords[0][1] + eye_coords[1][1]) / 2)
    top_left = (center_x - headset_width // 2, center_y - headset_height // 2)
    bottom_right = (center_x + headset_width // 2, center_y + headset_height // 2)

    # Draw the VR headset rectangle
    cv2.rectangle(image, top_left, bottom_right, (0, 0, 0), thickness=headset_thickness)

    return image

# Load the annotation data from the JSON file
with open(annotations_file, 'r') as f:
    annotations = json.load(f)

# Initialize a list to store augmented annotations
augmented_data = []

# Iterate through each annotation and process the corresponding image
for annotation in annotations['annotations']:
    image_id = annotation['image_id']
    
    # Extract the corresponding image metadata using the image_id
    image_info = next(img for img in annotations['images'] if img['id'] == image_id)
    image_path = os.path.join(dataset_path, image_info['file_name'])
    
    # Load the image
    image = cv2.imread(image_path)
    
    # Extract the eye coordinates from the keypoints (assuming keypoints 1 and 2 are the eyes)
    keypoints = annotation['keypoints']
    left_eye = [keypoints[3], keypoints[4]]
    right_eye = [keypoints[6], keypoints[7]]
    
    # Generate and overlay the VR headset
    augmented_image = generate_vr_headset(image, [left_eye, right_eye])
    
    # Save the augmented image
    output_image_path = os.path.join(output_path, image_info['file_name'])
    cv2.imwrite(output_image_path, augmented_image)
    
    # Store the augmented data (with the same annotation but the augmented image path)
    augmented_data.append({
        'image_id': image_id,
        'file_name': image_info['file_name'],
        'keypoints': keypoints
    })

# Save the updated annotations with references to the augmented images
augmented_annotations = {
    'images': annotations['images'],
    'annotations': augmented_data
}
with open(os.path.join(output_path, 'augmented_annotations.json'), 'w') as f:
    json.dump(augmented_annotations, f)

# Output the number of processed images
len(augmented_data)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Function to load the dataset
def load_pose_data(file_path, file_format='csv'):
    if file_format == 'csv':
        data = pd.read_csv(file_path)
    elif file_format == 'json':
        data = pd.read_json(file_path)
    elif file_format == 'npy':
        data = np.load(file_path)
    else:
        raise ValueError("Unsupported file format!")
    return data

# Function to preprocess the data (e.g., normalization)
def preprocess_data(data):
    # Assuming the data contains 2D keypoints as columns
    # Normalize the data (scaling between 0 and 1)
    scaler = StandardScaler()
    data_normalized = scaler.fit_transform(data)
    return data_normalized, scaler

# Function to split the data into training, validation, and testing sets
def split_data(data, test_size=0.2, validation_size=0.1):
    train_data, test_data = train_test_split(data, test_size=test_size, random_state=42)
    train_data, val_data = train_test_split(train_data, test_size=validation_size, random_state=42)
    return train_data, val_data, test_data

# Example usage
file_path = 'path_to_your_data.csv'  # Change this to your dataset path
pose_data = load_pose_data(file_path, file_format='csv')

# Preprocess the data
normalized_data, scaler = preprocess_data(pose_data)

# Split the data
train_data, val_data, test_data = split_data(normalized_data)

print("Training Data Shape:", train_data.shape)
print("Validation Data Shape:", val_data.shape)
print("Testing Data Shape:", test_data.shape)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the model architecture
def build_pose_conversion_model(input_shape):
    model = models.Sequential()
    
    # Input layer
    model.add(layers.InputLayer(input_shape=input_shape))
    
    # Hidden layers (Dense Layers)
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(32, activation='relu'))
    
    # Output layer
    # Assuming the output is a set of converted 2D keypoints
    model.add(layers.Dense(input_shape[0]))  # Output shape matches input shape
    
    # Compile the model
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    return model

# Example usage
input_shape = (34,)  # Change this according to the number of 2D keypoints (e.g., 17 keypoints x 2 for x, y coordinates)
pose_model = build_pose_conversion_model(input_shape)

pose_model.summary()


In [ ]:
# Function to train the model
def train_pose_model(model, train_data, val_data, epochs=50, batch_size=32):
    # Split features and labels (for pose conversion task)
    x_train, y_train = train_data[:, :input_shape[0]], train_data[:, input_shape[0]:]
    x_val, y_val = val_data[:, :input_shape[0]], val_data[:, input_shape[0]:]
    
    # Train the model
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1
    )
    
    return history

# Example usage
history = train_pose_model(pose_model, train_data, val_data)


In [ ]:
import numpy as np

# Function to compute MPJPE (Mean Per Joint Position Error)
def compute_mpjpe(pred_keypoints, true_keypoints):
    # Compute the Euclidean distance between predicted and true keypoints for each joint
    errors = np.linalg.norm(pred_keypoints - true_keypoints, axis=-1)
    
    # Calculate the mean error across all joints
    mpjpe = np.mean(errors)
    return mpjpe

# Function to compute MPJRE (Mean Per Joint Rotation Error)
def compute_mpjre(pred_rotations, true_rotations):
    # Assuming pred_rotations and true_rotations are in radians
    # Compute the angular difference between predicted and true rotations
    angle_diffs = np.arccos(np.clip(np.sum(pred_rotations * true_rotations, axis=-1), -1.0, 1.0))
    
    # Calculate the mean rotation error across all joints
    mpjre = np.mean(np.degrees(angle_diffs))
    return mpjre

# Function to evaluate the model using MPJPE and MPJRE
def evaluate_pose_model_with_metrics(model, test_data):
    x_test, y_test = test_data[:, :input_shape[0]], test_data[:, input_shape[0]:]
    
    # Predict keypoints
    pred_keypoints = model.predict(x_test)
    
    # Reshape keypoints if necessary (e.g., from (N, 34) to (N, 17, 2) for 17 keypoints in 2D)
    pred_keypoints = pred_keypoints.reshape(-1, 17, 2)
    true_keypoints = y_test.reshape(-1, 17, 2)
    
    # Compute MPJPE
    mpjpe = compute_mpjpe(pred_keypoints, true_keypoints)
    print("MPJPE:", mpjpe)
    
    # If rotation data is available, compute MPJRE (for 3D rotations)
    # Assuming y_test also contains rotation data for each joint
    pred_rotations = None  # Replace with actual prediction of joint rotations
    true_rotations = None  # Replace with actual ground truth joint rotations
    if pred_rotations is not None and true_rotations is not None:
        mpjre = compute_mpjre(pred_rotations, true_rotations)
        print("MPJRE:", mpjre)
    
    return mpjpe

# Example usage
mpjpe = evaluate_pose_model_with_metrics(pose_model, test_data)
